In [52]:
from mb_sql.conn import get_engine, get_session
from mb_sql.sql import read_sql
from mb_sql.utils import list_schemas,list_tables

In [53]:
engine = get_engine(name='postgresql' , db= 'postgres', user='postgres' , password= 'postgres', host= 'localhost', port= 5432, echo=False)
session = get_session(engine)

In [54]:
print(list_schemas(engine),f'\n\n',list_tables(engine, schema='public'))

    schema_name
0  embeddingsDB
1    pg_catalog
2      pg_toast
3        public
4      ytshorts 

        table_name
0     download_db
1  download_queue
2       merged_db
3    mix_video_db
4   transcript_db


In [55]:
read_sql('''SELECT
    column_name,
    data_type,
    character_maximum_length,
    is_nullable,
    column_default
FROM information_schema.columns
WHERE table_name = 'download_db'
ORDER BY ordinal_position;''', engine)

,column_name,data_type,character_maximum_length,is_nullable,column_default
0,yt_link,text,None,NO,None
1,local_link,text,None,YES,None
2,download,boolean,None,YES,None
3,date,date,None,YES,None
4,time,time without time zone,None,YES,None


In [56]:
read_sql('SELECT * FROM public.download_db limit 3;', engine)

,yt_link,local_link,download,date,time
0,https://www.youtube.com/watch?v=_nPQk65D3xE,/home/malav/Desktop/ytshorts_files/yt_download...,True,2025-05-23,02:04:29.359650
1,https://www.youtube.com/watch?v=_nPQk65D3xE,/home/malav/Desktop/ytshorts_files/yt_download...,True,2025-05-23,02:13:34.967209
2,https://www.youtube.com/watch?v=_nPQk65D3xE,/home/malav/Desktop/ytshorts_files/yt_download...,True,2025-05-23,02:16:29.363057


In [22]:
from mb_rag.agents.get_langsmith import set_langsmith_parameters

In [23]:
from mb_rag.utils.extra import load_dotenv
load_dotenv()

True

In [24]:
set_langsmith_parameters(langsmith_endpoint="https://api.smith.langchain.com",
                        langsmith_project="SQL-Agent-Project",
                        langsmith_tracing="true")